In [117]:
# 导入必要的库
import pandas as pd  # 用于数据处理
import numpy as np  # 用于数值计算
import json  # 用于处理JSON格式数据
# 绘图
import matplotlib.pyplot as plt  # 用于生成图表
from mplsoccer import VerticalPitch  # 专门用于足球比赛相关的图表绘制
# 统计模型拟合
import statsmodels.api as sm  # 提供广泛的统计模型
import statsmodels.formula.api as smf  # 通过公式形式拟合统计模型
# 打开数据文件
import os  # 用于操作系统相关的功能，如路径管理
import pathlib  # 用于更高层次的路径操作
import warnings  # 用于控制警告的显示

# 设置Pandas链式赋值模式为None，以忽略相关警告
pd.options.mode.chained_assignment = None
# 忽略警告信息
warnings.filterwarnings('ignore')

In [118]:
# 加载数据 - 存储到train数据框中
#/Users/max/MMF-HT24/Assignment2/data/events/events_European_Championship.json
train = pd.DataFrame()  # 初始化一个空的数据框
# 准备空的数据框
path = os.path.join(str(pathlib.Path().resolve()), 'data', 'events','events_World_Cup.json')  # 拼接数据文件路径

# 打开JSON文件并读取数据
with open(path) as f:  # 打开指定路径的JSON文件
    data = json.load(f)  # 加载JSON数据到变量data中

# 将数据转换为Pandas DataFrame
train = pd.DataFrame(data)

# 查看数据框的基本信息，如列名、数据类型和内存占用
train['index'] = train.index

In [119]:
shotsA = train.loc[train['eventId'] == 10]
shotsA

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,index
117,10,Shot,"[{'id': 402}, {'id': 2101}, {'id': 1802}]",122940,"[{'y': 27, 'x': 87}, {'y': 100, 'x': 100}]",2057954,Shot,16521,1H,327.142941,100,258612248,117
154,10,Shot,"[{'id': 402}, {'id': 2101}, {'id': 1802}]",101699,"[{'y': 69, 'x': 90}, {'y': 0, 'x': 0}]",2057954,Shot,14358,1H,407.123899,100,258612244,154
197,10,Shot,"[{'id': 401}, {'id': 2101}, {'id': 1802}]",101857,"[{'y': 30, 'x': 86}, {'y': 0, 'x': 0}]",2057954,Shot,14358,1H,526.276996,100,258612307,197
232,10,Shot,"[{'id': 101}, {'id': 403}, {'id': 201}, {'id':...",102157,"[{'y': 60, 'x': 93}, {'y': 0, 'x': 0}]",2057954,Shot,14358,1H,693.396917,100,258612368,232
372,10,Shot,"[{'id': 401}, {'id': 2101}, {'id': 1802}]",122671,"[{'y': 38, 'x': 86}, {'y': 100, 'x': 100}]",2057954,Shot,16521,1H,1266.276267,100,258612558,372
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101371,10,Shot,"[{'id': 101}, {'id': 402}, {'id': 201}, {'id':...",14943,"[{'y': 53, 'x': 92}, {'y': 100, 'x': 100}]",2058017,Shot,9598,2H,1389.182669,100,263885208,101371
101457,10,Shot,"[{'id': 402}, {'id': 201}, {'id': 1211}, {'id'...",69409,"[{'y': 61, 'x': 65}, {'y': 100, 'x': 100}]",2058017,Shot,9598,2H,1802.299261,100,263885299,101457
101493,10,Shot,"[{'id': 401}, {'id': 2101}, {'id': 1802}]",3476,"[{'y': 33, 'x': 81}, {'y': 100, 'x': 100}]",2058017,Shot,9598,2H,1933.996341,100,263885348,101493
101620,10,Shot,"[{'id': 401}, {'id': 201}, {'id': 1204}, {'id'...",28115,"[{'y': 76, 'x': 78}, {'y': 0, 'x': 0}]",2058017,Shot,4418,2H,2487.443304,100,263885485,101620


In [120]:
Ppre_shots = train.loc[shotsA['index']-1]
Ppre_shots

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,index
116,8,Simple pass,[{'id': 1801}],122832,"[{'y': 22, 'x': 71}, {'y': 27, 'x': 87}]",2057954,Pass,16521,1H,325.272369,85,258612246,116
153,8,Cross,"[{'id': 402}, {'id': 1801}]",41123,"[{'y': 79, 'x': 95}, {'y': 69, 'x': 90}]",2057954,Pass,14358,1H,406.080614,80,258612233,153
196,1,Ground loose ball duel,"[{'id': 703}, {'id': 1801}]",101857,"[{'y': 24, 'x': 81}, {'y': 30, 'x': 86}]",2057954,Duel,14358,1H,524.214132,13,258612306,196
231,8,Cross,"[{'id': 301}, {'id': 402}, {'id': 801}, {'id':...",257800,"[{'y': 23, 'x': 77}, {'y': 60, 'x': 93}]",2057954,Pass,14358,1H,690.869877,80,258612366,231
371,8,Head pass,"[{'id': 1401}, {'id': 1802}]",103668,"[{'y': 65, 'x': 7}, {'y': 62, 'x': 14}]",2057954,Pass,14358,1H,1263.199829,82,258612574,371
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101370,1,Ground defending duel,"[{'id': 501}, {'id': 703}, {'id': 1801}]",14943,"[{'y': 54, 'x': 94}, {'y': 53, 'x': 92}]",2058017,Duel,9598,2H,1388.157139,12,263885258,101370
101456,8,Simple pass,[{'id': 1801}],69968,"[{'y': 20, 'x': 57}, {'y': 61, 'x': 65}]",2058017,Pass,9598,2H,1798.035328,85,263885297,101456
101492,7,Touch,[],69411,"[{'y': 29, 'x': 90}, {'y': 33, 'x': 81}]",2058017,Others on the ball,9598,2H,1932.975578,72,263885346,101492
101619,1,Ground defending duel,"[{'id': 501}, {'id': 701}, {'id': 1802}]",69968,"[{'y': 14, 'x': 23}, {'y': 24, 'x': 22}]",2058017,Duel,9598,2H,2484.797699,12,263885709,101619


In [121]:
# 获取射门位置坐标并分别存储在X和Y列中
shotsA["AX"] = shotsA.positions.apply(lambda cell: (100 - cell[0]['x']) * 105/100)  # 计算射门的X坐标并转换为球场尺度（105米）
shotsA["AY"] = shotsA.positions.apply(lambda cell: cell[0]['y'] * 68/100)  # 计算射门的Y坐标并转换为球场尺度（68米）
shotsA["AC"] = shotsA.positions.apply(lambda cell: abs(cell[0]['y'] - 50) * 68/100)  # 计算射门与球场中线（Y=50）之间的垂直距离

# 计算射门距离和角度
shotsA["ADistance"] = np.sqrt(shotsA["AX"]**2 + shotsA["AC"]**2)  # 使用勾股定理计算射门距离
shotsA["AAngle"] = np.where(np.arctan(7.32 * shotsA["AX"] / (shotsA["AX"]**2 + shotsA["AC"]**2 - (7.32/2)**2)) > 0,
                          np.arctan(7.32 * shotsA["AX"] /(shotsA["AX"]**2 + shotsA["AC"]**2 - (7.32/2)**2)),
                          np.arctan(7.32 * shotsA["AX"] /(shotsA["AX"]**2 + shotsA["AC"]**2 - (7.32/2)**2)) + np.pi)
# 计算射门角度（使用球门的宽度7.32米）

# 如果在建模过程中遇到依赖变量0和1颠倒的问题，可以将目标变量转换为object类型
shotsA["Goal"] = shotsA.tags.apply(lambda x: 1 if {'id':101} in x else 0).astype(object)  # 标记是否进球，'id':101表示进球事件 id = 301 为assist(助攻)

In [122]:
shotsA
sub_A = ['index','AX',	'AY',	'AC',	'ADistance',	'AAngle'	,'Goal']
sub_shotsA = shotsA[sub_A]
sub_shotsA['index'] = sub_shotsA['index'] -1
sub_shotsA

,index,AX,AY,AC,ADistance,AAngle,Goal
117,116,13.65,18.36,15.64,20.758904,0.234886,0
154,153,10.50,46.92,12.92,16.648616,0.283528,0
197,196,14.70,20.40,13.60,20.026233,0.270761,0
232,231,7.35,40.80,6.80,10.013116,0.554534,1
372,371,14.70,25.84,8.16,16.812959,0.380161,0
...,...,...,...,...,...,...,...
101371,101370,8.40,36.04,2.04,8.644166,0.786717,1
101457,101456,36.75,41.48,7.48,37.503505,0.190752,0
101493,101492,19.95,22.44,11.56,23.057235,0.274666,0
101620,101619,23.10,51.68,17.68,29.089386,0.200318,0


In [123]:
# 获取射门位置坐标并分别存储在X和Y列中
Ppre_shots["PX"] = Ppre_shots.positions.apply(lambda cell: (100 - cell[0]['x']) * 105/100)  # 计算射门的X坐标并转换为球场尺度（105米）
Ppre_shots["PY"] = Ppre_shots.positions.apply(lambda cell: cell[0]['y'] * 68/100)  # 计算射门的Y坐标并转换为球场尺度（68米）
Ppre_shots["PC"] = Ppre_shots.positions.apply(lambda cell: abs(cell[0]['y'] - 50) * 68/100)  # 计算射门与球场中线（Y=50）之间的垂直距离

# 计算射门距离和角度
Ppre_shots["PDistance"] = np.sqrt(Ppre_shots["PX"]**2 + Ppre_shots["PC"]**2)  # 使用勾股定理计算射门距离
Ppre_shots["PAngle"] = np.where(np.arctan(7.32 * Ppre_shots["PX"] / (Ppre_shots["PX"]**2 + Ppre_shots["PC"]**2 - (7.32/2)**2)) > 0,
                          np.arctan(7.32 * Ppre_shots["PX"] /(Ppre_shots["PX"]**2 + Ppre_shots["PC"]**2 - (7.32/2)**2)),
                          np.arctan(7.32 * Ppre_shots["PX"] /(Ppre_shots["PX"]**2 + Ppre_shots["PC"]**2 - (7.32/2)**2)) + np.pi)
# 计算射门角度（使用球门的宽度7.32米）

In [124]:
Ppre_shots 

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,index,PX,PY,PC,PDistance,PAngle
116,8,Simple pass,[{'id': 1801}],122832,"[{'y': 22, 'x': 71}, {'y': 27, 'x': 87}]",2057954,Pass,16521,1H,325.272369,85,258612246,116,30.45,14.96,19.04,35.912729,0.172893
153,8,Cross,"[{'id': 402}, {'id': 1801}]",41123,"[{'y': 79, 'x': 95}, {'y': 69, 'x': 90}]",2057954,Pass,14358,1H,406.080614,80,258612233,153,5.25,53.72,19.72,20.406884,0.095062
196,1,Ground loose ball duel,"[{'id': 703}, {'id': 1801}]",101857,"[{'y': 24, 'x': 81}, {'y': 30, 'x': 86}]",2057954,Duel,14358,1H,524.214132,13,258612306,196,19.95,16.32,17.68,26.656798,0.206476
231,8,Cross,"[{'id': 301}, {'id': 402}, {'id': 801}, {'id':...",257800,"[{'y': 23, 'x': 77}, {'y': 60, 'x': 93}]",2057954,Pass,14358,1H,690.869877,80,258612366,231,24.15,15.64,18.36,30.336646,0.192508
371,8,Head pass,"[{'id': 1401}, {'id': 1802}]",103668,"[{'y': 65, 'x': 7}, {'y': 62, 'x': 14}]",2057954,Pass,14358,1H,1263.199829,82,258612574,371,97.65,44.20,10.20,98.181274,0.074120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101370,1,Ground defending duel,"[{'id': 501}, {'id': 703}, {'id': 1801}]",14943,"[{'y': 54, 'x': 94}, {'y': 53, 'x': 92}]",2058017,Duel,9598,2H,1388.157139,12,263885258,101370,6.30,36.72,2.72,6.862099,0.939821
101456,8,Simple pass,[{'id': 1801}],69968,"[{'y': 20, 'x': 57}, {'y': 61, 'x': 65}]",2058017,Pass,9598,2H,1798.035328,85,263885297,101456,45.15,13.60,20.40,49.544752,0.134561
101492,7,Touch,[],69411,"[{'y': 29, 'x': 90}, {'y': 33, 'x': 81}]",2058017,Others on the ball,9598,2H,1932.975578,72,263885346,101492,10.50,19.72,14.28,17.724796,0.250188
101619,1,Ground defending duel,"[{'id': 501}, {'id': 701}, {'id': 1802}]",69968,"[{'y': 14, 'x': 23}, {'y': 24, 'x': 22}]",2058017,Duel,9598,2H,2484.797699,12,263885709,101619,80.85,9.52,24.48,84.474806,0.082900


In [125]:
MergeA_B_endshot = pd.merge(Ppre_shots, sub_shotsA, on='index')
MergeA_B_endshot

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,...,PY,PC,PDistance,PAngle,AX,AY,AC,ADistance,AAngle,Goal
0,8,Simple pass,[{'id': 1801}],122832,"[{'y': 22, 'x': 71}, {'y': 27, 'x': 87}]",2057954,Pass,16521,1H,325.272369,...,14.96,19.04,35.912729,0.172893,13.65,18.36,15.64,20.758904,0.234886,0
1,8,Cross,"[{'id': 402}, {'id': 1801}]",41123,"[{'y': 79, 'x': 95}, {'y': 69, 'x': 90}]",2057954,Pass,14358,1H,406.080614,...,53.72,19.72,20.406884,0.095062,10.50,46.92,12.92,16.648616,0.283528,0
2,1,Ground loose ball duel,"[{'id': 703}, {'id': 1801}]",101857,"[{'y': 24, 'x': 81}, {'y': 30, 'x': 86}]",2057954,Duel,14358,1H,524.214132,...,16.32,17.68,26.656798,0.206476,14.70,20.40,13.60,20.026233,0.270761,0
3,8,Cross,"[{'id': 301}, {'id': 402}, {'id': 801}, {'id':...",257800,"[{'y': 23, 'x': 77}, {'y': 60, 'x': 93}]",2057954,Pass,14358,1H,690.869877,...,15.64,18.36,30.336646,0.192508,7.35,40.80,6.80,10.013116,0.554534,1
4,8,Head pass,"[{'id': 1401}, {'id': 1802}]",103668,"[{'y': 65, 'x': 7}, {'y': 62, 'x': 14}]",2057954,Pass,14358,1H,1263.199829,...,44.20,10.20,98.181274,0.074120,14.70,25.84,8.16,16.812959,0.380161,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1,Ground defending duel,"[{'id': 501}, {'id': 703}, {'id': 1801}]",14943,"[{'y': 54, 'x': 94}, {'y': 53, 'x': 92}]",2058017,Duel,9598,2H,1388.157139,...,36.72,2.72,6.862099,0.939821,8.40,36.04,2.04,8.644166,0.786717,1
1415,8,Simple pass,[{'id': 1801}],69968,"[{'y': 20, 'x': 57}, {'y': 61, 'x': 65}]",2058017,Pass,9598,2H,1798.035328,...,13.60,20.40,49.544752,0.134561,36.75,41.48,7.48,37.503505,0.190752,0
1416,7,Touch,[],69411,"[{'y': 29, 'x': 90}, {'y': 33, 'x': 81}]",2058017,Others on the ball,9598,2H,1932.975578,...,19.72,14.28,17.724796,0.250188,19.95,22.44,11.56,23.057235,0.274666,0
1417,1,Ground defending duel,"[{'id': 501}, {'id': 701}, {'id': 1802}]",69968,"[{'y': 14, 'x': 23}, {'y': 24, 'x': 22}]",2058017,Duel,9598,2H,2484.797699,...,9.52,24.48,84.474806,0.082900,23.10,51.68,17.68,29.089386,0.200318,0


In [126]:
MergeA_B_endshot['PADistance'] = np.sqrt((MergeA_B_endshot['PX'] - MergeA_B_endshot['AX'])**2 + (MergeA_B_endshot['PY'] - MergeA_B_endshot['AY'])**2)
MergeA_B_endshot['PAAngle'] = np.arctan2(MergeA_B_endshot['PY'] - MergeA_B_endshot['AY'], MergeA_B_endshot['PX'] - MergeA_B_endshot['AX'])

In [127]:
MergeA_B_endshot['Goal'] = pd.to_numeric(MergeA_B_endshot['Goal'], errors='coerce')
MergeA_B_endshot.loc[0]

eventId                                                8
subEventName                                 Simple pass
tags                                      [{'id': 1801}]
playerId                                          122832
positions       [{'y': 22, 'x': 71}, {'y': 27, 'x': 87}]
matchId                                          2057954
eventName                                           Pass
teamId                                             16521
matchPeriod                                           1H
eventSec                                      325.272369
subEventId                                            85
id                                             258612246
index                                                116
PX                                                 30.45
PY                                                 14.96
PC                                                 19.04
PDistance                                      35.912729
PAngle                         

In [128]:
features = ['PX','PY','PC','PDistance','PAngle','AX','AY','AC','ADistance','AAngle','PADistance','PAngle']
target = ['Goal']

In [129]:
nn_feature = MergeA_B_endshot[features]
nn_target = MergeA_B_endshot[target]

In [130]:
nn_feature

,PX,PY,PC,PDistance,PAngle,AX,AY,AC,ADistance,AAngle,PADistance,PAngle
0,30.45,14.96,19.04,35.912729,0.172893,13.65,18.36,15.64,20.758904,0.234886,17.140595,0.172893
1,5.25,53.72,19.72,20.406884,0.095062,10.50,46.92,12.92,16.648616,0.283528,8.590838,0.095062
2,19.95,16.32,17.68,26.656798,0.206476,14.70,20.40,13.60,20.026233,0.270761,6.648977,0.206476
3,24.15,15.64,18.36,30.336646,0.192508,7.35,40.80,6.80,10.013116,0.554534,30.253357,0.192508
4,97.65,44.20,10.20,98.181274,0.074120,14.70,25.84,8.16,16.812959,0.380161,84.957590,0.074120
...,...,...,...,...,...,...,...,...,...,...,...,...
1414,6.30,36.72,2.72,6.862099,0.939821,8.40,36.04,2.04,8.644166,0.786717,2.207351,0.939821
1415,45.15,13.60,20.40,49.544752,0.134561,36.75,41.48,7.48,37.503505,0.190752,29.117939,0.134561
1416,10.50,19.72,14.28,17.724796,0.250188,19.95,22.44,11.56,23.057235,0.274666,9.833662,0.250188
1417,80.85,9.52,24.48,84.474806,0.082900,23.10,51.68,17.68,29.089386,0.200318,71.501945,0.082900


In [131]:
nn_target

,Goal
0,0
1,0
2,0
3,1
4,0
...,...
1414,1
1415,0
1416,0
1417,0


In [132]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


# 分离特征和目标变量
features = MergeA_B_endshot[['PX', 'PY', 'PC', 'PDistance', 'PAngle', 'AX', 'AY', 'AC', 'ADistance', 'AAngle', 'PADistance', 'PAngle']]
target = MergeA_B_endshot['Goal']

# 标准化特征
scaler = StandardScaler()
features = scaler.fit_transform(features)

# 转换为 PyTorch 张量
features = torch.tensor(features, dtype=torch.float32)
target = torch.tensor(target.values.astype(np.float32)).unsqueeze(1)

# 创建数据集和加载器
dataset = TensorDataset(features, target)
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10)
test_loader = DataLoader(test_dataset, batch_size=10)

# 模型定义
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(12, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

model = NeuralNet()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [133]:
# 确保模型加载并处于评估模式
model = NeuralNet()
model.load_state_dict(torch.load('xA_model.pth'))
model.eval()  # 设置为评估模式

# 获取 test_data_f 中的特征部分 (与训练时的特征相同)
test_features = MergeA_B_endshot[['PX', 'PY', 'PC', 'PDistance', 'PAngle', 'AX', 'AY', 'AC', 'ADistance', 'AAngle', 'PADistance', 'PAngle']]

# 标准化测试集的特征
test_features_scaled = scaler.transform(test_features)

# 转换为 PyTorch 张量
test_features_tensor = torch.tensor(test_features_scaled, dtype=torch.float32)

# 存储预测概率
predicted_probabilities = []

# 对测试集数据进行预测
with torch.no_grad():
    for data in test_features_tensor:
        probability = model(data.unsqueeze(0)).item()  # 添加 batch 维度
        predicted_probabilities.append(probability)

# 检查预测概率和原始数据的长度是否匹配
print(f"Predicted probabilities length: {len(predicted_probabilities)}")
print(f"Test data length: {len(MergeA_B_endshot)}")

# 确保两者长度相等
if len(predicted_probabilities) == len(MergeA_B_endshot):
    # 将预测的概率值添加到 test_data_f['xA'] 列中
    MergeA_B_endshot['xA'] = predicted_probabilities
else:
    print("Error: Length mismatch between predicted probabilities and test data.")

# 输出添加了预测概率后的 DataFrame
MergeA_B_endshot.loc[0]  # 只打印前几行作为示例


Predicted probabilities length: 1419
Test data length: 1419


eventId                                                8
subEventName                                 Simple pass
tags                                      [{'id': 1801}]
playerId                                          122832
positions       [{'y': 22, 'x': 71}, {'y': 27, 'x': 87}]
matchId                                          2057954
eventName                                           Pass
teamId                                             16521
matchPeriod                                           1H
eventSec                                      325.272369
subEventId                                            85
id                                             258612246
index                                                116
PX                                                 30.45
PY                                                 14.96
PC                                                 19.04
PDistance                                      35.912729
PAngle                         

In [134]:
player_xA_totals = MergeA_B_endshot.groupby('playerId')['xA'].sum().reset_index()

In [135]:
player_xA_totals_sorted = player_xA_totals.sort_values(by='xA', ascending=False).reset_index(drop=True)
player_xA_totals_sorted[:6]

,playerId,xA
0,3359,2.432224
1,38021,2.166653
2,61949,2.079575
3,40810,1.941605
4,3431,1.570940
5,7936,1.331450


In [136]:
# 加载数据 - 存储到train数据框中
engplayers = pd.DataFrame()  # 初始化一个空的数据框
# 准备空的数据框
path = os.path.join(str(pathlib.Path().resolve()), 'data','players.json')  # 拼接数据文件路径

# 打开JSON文件并读取数据
with open(path) as f:  # 打开指定路径的JSON文件
    data = json.load(f)  # 加载JSON数据到变量data中

# 将数据转换为Pandas DataFrame
engplayers = pd.DataFrame(data)

# 查看数据框的基本信息，如列名、数据类型和内存占用
engplayers.loc[0]

passportArea             {'name': 'Turkey', 'id': '792', 'alpha3code': ...
weight                                                                  78
firstName                                                            Harun
middleName                                                                
lastName                                                             Tekin
currentTeamId                                                         4502
birthDate                                                       1989-06-17
height                                                                 187
role                     {'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...
birthArea                {'name': 'Turkey', 'id': '792', 'alpha3code': ...
wyId                                                                 32777
foot                                                                 right
shortName                                                         H. Tekin
currentNationalTeamId    

In [137]:
import pandas as pd

# Assuming 'player_xA_totals_sorted' and 'engplayers' are already defined DataFrames

# Merging the DataFrames on the specified keys
engplayers_with_xA = pd.merge(engplayers, player_xA_totals_sorted, 
                              left_on='wyId', right_on='playerId', how='right')
# Display the merged DataFrame
engplayers_with_xA.loc[:6]

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId,playerId,xA
0,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",72,Lionel Andr\u00e9s,,Messi Cuccittini,676,1987-06-24,170,"{'code2': 'FW', 'code3': 'FWD', 'name': 'Forwa...","{'name': 'Argentina', 'id': '32', 'alpha3code'...",3359,left,L. Messi,12274,3359,2.432224
1,"{'name': 'Belgium', 'id': '56', 'alpha3code': ...",68,Kevin,,De Bruyne,1625,1991-06-28,181,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Belgium', 'id': '56', 'alpha3code': ...",38021,right,K. De Bruyne,5629,38021,2.166653
2,"{'name': 'Sweden', 'id': 752, 'alpha3code': 'S...",77,Mikael,,Lustig,8508,1986-12-13,189,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Sweden', 'id': 752, 'alpha3code': 'S...",61949,right,M. Lustig,7047,61949,2.079575
3,"{'name': 'Brazil', 'id': '76', 'alpha3code': '...",68,Neymar,,da Silva Santos Junior,3767,1992-02-05,175,"{'code2': 'FW', 'code3': 'FWD', 'name': 'Forwa...","{'name': 'Brazil', 'id': '76', 'alpha3code': '...",40810,right,Neymar,6380,40810,1.941605
4,"{'name': 'Brazil', 'id': '76', 'alpha3code': '...",76,Jo\u00e3o,,Miranda de Souza Filho,3161,1984-09-07,186,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Brazil', 'id': '76', 'alpha3code': '...",3431,right,Jo\u00e3o Miranda,6380,3431,1.570940
5,"{'name': 'Guinea', 'id': '324', 'alpha3code': ...",84,Paul,,Pogba,1611,1993-03-15,191,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",7936,right,P. Pogba,4418,7936,1.331450
6,"{'name': 'Croatia', 'id': '191', 'alpha3code':...",76,\u0160ime,,Vrsaljko,3161,1992-01-10,181,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Croatia', 'id': '191', 'alpha3code':...",69409,right,\u0160. Vrsaljko,9598,69409,1.325700
